In [ ]:
import re
import os
import gzip
import requests
from lxml import html
from Bio import SeqIO

In [ ]:
#Enter your clan ID
clan_id = "CL0023"

#Setting the absolute clan url
clan_url = "http://pfam.xfam.org/clan/"+clan_id

#Getting the HTML and building the tree to query
response = requests.get(clan_url)
tree = html.fromstring(response.content)

#Get all pfam id and names
pfam_id = tree.xpath('//span[@class="listItem"]/a/@title')
pfam_names = tree.xpath('//span[@class="listItem"]/a/text()')
clan_families_info = [[re.sub(r"\W","",re.sub(r"\n","",x)) , y] for x, y in zip(pfam_names, pfam_id)] #Removing spaces and endlines



In [ ]:
#Database alignments to download
db_to_use = "uniprot"
path_to_save = "/home/roc/workspace/I-have-time/pfam/"
if not os.path.exists(path_to_save):
    os.makedirs(path_to_save)

i = 0
for family in clan_families_info:
    print(i)
    i+=1
    url = "http://pfam.xfam.org/family/"+family[1]+"/alignment/"+db_to_use+"/gzipped"
    os.system("wget "+url+" -O "+path_to_save+family[1]+".gz")

In [ ]:
#listing all files
pfam_files = os.listdir(path_to_save)

#Opening the output file
f_w = open(path_to_save+'all-id.txt', "w+")
i=0

#Iterating through all families
for _file in pfam_files:
    if _file == 'all-id.txt': continue #Preventing mistakes
    i+=1
    print(i, _file)
    with gzip.open(path_to_save+_file, 'rb') as f:
        temp_id = set([record.id.split('.')[0] for record in SeqIO.parse(f, "stockholm")]) #Fetching all ids from the file
        for elm in temp_id:
            f_w.write(elm+'\n')

            

